# villo application of attribute scoring
- Environment: colab pro

## Reference
- [SCORE_REPRO.md](https://github.com/Muennighoff/vilio/blob/master/SCORE_REPRO.md)

## Prerequisite (Google Drive)
- Add kaggle.json to MyDrive/vilio
- Add hateful memes challenge data as MyDrive/vilio/hateful_memes.zip
- Add confounders to MyDrive/annotation
    - See `benign_confounder.ipynb` for annotation details

## Protocol
### Prerequisite
- Dependent on which model to run, comment in/out codes in `init / data`, `additional installation`, `inference` subsections

### inference-only
- Run till `additional installation > check gpu version`
  - Just `Run all` should be fine - will stop automatically
- Run the rest
  - Rerun it if stopped in `check gpu version`



## init

In [ ]:
import time
t0 = time.time()

In [ ]:
# mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model = "V"#["O","U","V","E"]

In [ ]:
if model in ["O","U","V"]:
  data_path = "/vilio/data/"
else:#E
  data_path = "/vilio/ernie-vil/data/hm/"

### installation

In [ ]:
# vilio basics
!cd ../; git clone -b master https://github.com/Muennighoff/vilio.git
# !cd ../vilio/py-bottom-up-attention; pip install -r requirements.txt
# !pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
# !cd ../vilio/py-bottom-up-attention; python setup.py build develop

Cloning into 'vilio'...
remote: Enumerating objects: 2838, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 2838 (delta 0), reused 1 (delta 0), pack-reused 2834
Receiving objects: 100% (2838/2838), 10.63 MiB | 16.05 MiB/s, done.
Resolving deltas: 100% (1492/1492), done.


In [ ]:
# setup clone-anonymous-github
!git clone https://github.com/fedebotu/clone-anonymous-github
!cd clone-anonymous-github; pip install -r requirements.txt

In [ ]:
# main repo
!cd clone-anonymous-github; python src/download.py \
    --url https://anonymous.4open.science/r/MemesModalityEvaluation-2540 \
    --save_dir /

In [ ]:
# replace / add required files
!cd /MemesModalityEvaluation-2540/shell; cp vilio_overwrite_scripts.sh /content
!bash ./vilio_overwrite_scripts.sh
!rm -r /MemesModalityEvaluation-2540

In [ ]:
# for lmdb feats
!pip install -q kaggle

In [ ]:
# added for eps
!pip install kaleido
!apt-get install poppler-utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 22.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 174 kB of archives.
After this operation, 754 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.1 [174 kB]
Fetched 174 kB in 1s (137 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 122518 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.86.1-0ubuntu1.1_amd64.deb ...
Unpacking poppler-utils (0.86.1-0ubuntu1.1) ...
Setting up poppler-utils (0.86.1-0ubuntu1.1) ...
Processing triggers for man-db (2.9.1-1) ...


### kaggle setup
- [reference](https://www.kaggle.com/general/74235)

In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/vilio/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

ref                                                                title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
arnabchaki/data-science-salaries-2023                              Data Science Salaries 2023 💸                       25KB  2023-04-13 09:55:16          12115        318  1.0              
asahu40/walmart-data-analysis-and-forcasting                       Walmart Data Analysis and Forcasting              122KB  2023-04-26 07:07:03           1793         40  1.0              
ahmedshahriarsakib/usa-real-estate-dataset                         USA Real Estate Dataset                             1MB  2023-04-30 03:51:36           1453         41  1.0              
iammustafatz/diabetes-prediction-dataset               

### data
- [kaggle reference](https://qiita.com/k_ikasumipowder/items/1c20d8b68dbc94ab2633)

In [ ]:
%%capture
# hateful memes from MyDrive
!mkdir tmp_data
!unzip /content/drive/MyDrive/vilio/hateful_memes.zip -d tmp_data


In [ ]:
# hateful memes from MyDrive
!cp -r tmp_data/hateful_memes/* $data_path

In [ ]:
# kaggle lmdb
!kaggle datasets download muennighoff/hmfeatureszipfin

100% 9.58G/9.58G [08:15<00:00, 20.7MB/s]
100% 9.58G/9.58G [08:15<00:00, 20.7MB/s]


In [ ]:
# kaggle lmdb
!unzip hmfeatureszipfin.zip -d $data_path

Archive:  hmfeatureszipfin.zip
  inflating: /vilio/data/detectron.lmdb/data.mdb  
  inflating: /vilio/data/detectron.lmdb/lock.mdb  


In [ ]:
## delete original for memory saving
!rm -r tmp_data

In [ ]:
# kaggle lmdb
## delete original for memory saving
!rm hmfeatureszipfin.zip

In [ ]:
# confounders
# !mkdir /vilio/data
!cp /content/drive/MyDrive/annotation/confounders.parquet /vilio/data

## additional installation

### for torch 1.6.0
- [cuda 10.2](https://gist.github.com/tzvsi/222b3b22a847004a729744f89fe31255)
- [python 3.6](https://stackoverflow.com/questions/66775948/downgrade-python-version-from-3-7-to-3-6-in-google-colab)

#### python version

In [ ]:
%%bash

# MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_INSTALLER_SCRIPT=Miniconda3-py37_23.1.0-1-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
Unpacking payload ...

Installing base environment...





Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /usr/local


--2023-05-12 01:37:09--  https://repo.continuum.io/miniconda/Miniconda3-py37_23.1.0-1-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-py37_23.1.0-1-Linux-x86_64.sh [following]
--2023-05-12 01:37:09--  https://repo.anaconda.com/miniconda/Miniconda3-py37_23.1.0-1-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90665082 (86M) [application/x-sh]
Saving to: ‘Miniconda3-py37_23.1.0-1-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 44.6M 2s
    50K .......... .......... ....

In [ ]:
import sys
# _ = (sys.path.append("/usr/local/lib/python3.6/site-packages"))
_ = (sys.path.append("/usr/local/lib/python3.7/site-packages"))

In [ ]:
!python --version
!python3 --version

Python 3.7.16
Python 3.7.16


#### cuda 10.2

In [ ]:
!mkdir install ; cd install
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!add-apt-repository "deb http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/ /"
!apt-get update
!apt-get -y install cuda-10-2

--2023-05-12 01:37:18--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.39.144
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu1804.pin’

cuda-ubuntu1804.pin 100%[===================>]     190  --.-KB/s    in 0s      

2023-05-12 01:37:18 (8.30 MB/s) - ‘cuda-ubuntu1804.pin’ saved [190/190]

Executing: /tmp/apt-key-gpghome.GCuuVPtE1b/gpg.1.sh --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
gpg: requesting key from 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub'
gpg: key F60F4B3D7FA2AF80: public key "cudatools <cudatools@nvidia.com>" imported
gpg: Total number processed: 1
gpg:               import

In [ ]:
!ls -d /usr/local/cuda-*
!which nvcc

/usr/local/cuda-10.2  /usr/local/cuda-11  /usr/local/cuda-11.8
/usr/local/cuda/bin/nvcc


In [ ]:
import os
p = os.getenv('PATH')
ld = os.getenv('LD_LIBRARY_PATH')
os.environ['PATH'] = f"/usr/local/cuda-10.2/bin:{p}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-10.2/lib64:{ld}"
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Oct_23_19:24:38_PDT_2019
Cuda compilation tools, release 10.2, V10.2.89


#### O/U/V

In [ ]:
%%writefile new_req.txt
sacremoses==0.0.43
pandas==1.1.3
regex==2020.4.4
h5py==2.10.0
filelock==3.0.10
scipy==1.4.1
sentencepiece~=0.1.91
matplotlib==3.2.1
tensorflow==2.3.1
tqdm==4.45.0
numpy==1.18.1
six==1.14.0
packaging==20.1
wandb==0.10.8
psutil==5.7.0
requests==2.23.0
pytorch_lightning==1.0.4
ImageHash==4.1.0
tokenizers~=0.9.2
transformers==3.5.1 # Required due to some imports in the files under src/vilio/transformers
torchvision==0.7.0
jieba==0.42.1
botocore==1.19.8
spacy==2.3.2
boto3==1.16.8
comet_ml==3.2.5
dataclasses==0.6
fairseq==0.9.0
ftfy==5.8
fugashi==1.0.5
ipadic==1.0.0
lmdb==1.0.0
Pillow==8.0.1
py3nvml==0.2.6
pydantic==1.7.2
pythainlp==2.2.4
PyYAML==5.3.1
scikit_learn==0.23.2
tensorboardX==2.1
timeout_decorator==0.4.1
torchcontrib==0.0.2

torch==1.6.0

Writing new_req.txt


In [ ]:
!pip install -r new_req.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.8/883.8 kB 43.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.2/679.2 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 M

### for this module

In [ ]:
%%writefile grad_req.txt
einops==0.6.0
dask==2022.2.0
plotly==5.13.1
pyarrow==11.0.0
kaleido==0.2.1
ipykernel==5.5.6
cloudpickle==2.2.1
IPython==7.34.0
transformers==3.5.1
imgkit==1.2.3

Writing grad_req.txt


In [ ]:
!pip install --no-deps -r grad_req.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/35.1 MB 47.9 MB/s eta 0:00:00
  Using cached kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl (79.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.8/793.8 kB 60.9 MB/s eta 0:00:00


In [ ]:
!apt-get update && apt-get install -y wkhtmltopdf && apt-get clean

Hit:1 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The followi

## inference

### download extracted features

In [ ]:
model = "O"#["O","U","V","E"]

In [ ]:
# redefine data_path for session restart
if model in ["O","U","V"]:
  data_path = "/vilio/data/"
else:#E
  data_path = "/vilio/ernie-vil/data/hm/"

In [ ]:
# tsv features
!kaggle datasets download muennighoff/hmtsvfeats

100% 18.2G/18.2G [15:37<00:00, 24.5MB/s]
100% 18.2G/18.2G [15:37<00:00, 20.8MB/s]


In [ ]:
# tsv features
!unzip hmtsvfeats.zip -d $data_path
## delete original for memory saving
!rm hmtsvfeats.zip

Archive:  hmtsvfeats.zip
  inflating: /vilio/data/hm_vg10100.tsv  
  inflating: /vilio/data/hm_vg5050.tsv  
  inflating: /vilio/data/hm_vgattr10100.tsv  
  inflating: /vilio/data/hm_vgattr3636.tsv  
  inflating: /vilio/data/hm_vgattr5050.tsv  
  inflating: /vilio/data/hm_vgattr7272.tsv  


In [ ]:
# check list of data
!echo $data_path
!echo =========================
!cd ..$data_path; ls

/vilio/data/
confounders.parquet  hm_vg10100.tsv	 hm_vgattr5050.tsv  test_seen.jsonl
detectron.lmdb	     hm_vg5050.tsv	 hm_vgattr7272.tsv  test_unseen.jsonl
dev_seen.jsonl	     hm_vgattr10100.tsv  img		    train.jsonl
dev_unseen.jsonl     hm_vgattr3636.tsv	 README.md


### download pretrained/fine-tuned model

In [ ]:
model_dir = "/vilio/ckpt"
!mkdir $model_dir
# # O
# !mkdir $model_dir/vilioo36;mkdir $model_dir/vilioo50;mkdir $model_dir/vilioov50
# !kaggle datasets download muennighoff/vilioo36
# !unzip -j vilioo36.zip -d $model_dir/vilioo36
# !rm vilioo36.zip
# !kaggle datasets download muennighoff/vilioo50
# !unzip -j vilioo50.zip -d $model_dir/vilioo50
# !rm vilioo50.zip
# !kaggle datasets download muennighoff/vilioov50
# !unzip -j vilioov50.zip -d $model_dir/vilioov50
# !rm vilioov50.zip
# # U
# !mkdir $model_dir/viliou36;mkdir $model_dir/viliou50;mkdir $model_dir/viliou72
# !kaggle datasets download muennighoff/viliou36
# !unzip -j viliou36.zip -d $model_dir/viliou36
# !rm viliou36.zip
# !kaggle datasets download muennighoff/viliou50
# !unzip -j viliou50.zip -d $model_dir/viliou50
# !rm viliou50.zip
# !kaggle datasets download muennighoff/viliou72
# !unzip -j viliou72.zip -d $model_dir/viliou72
# !rm viliou72.zip
# V
!mkdir $model_dir/viliov45;mkdir $model_dir/viliov90;mkdir $model_dir/viliov135
!kaggle datasets download muennighoff/viliov45
!unzip -j viliov45.zip -d $model_dir/viliov45
!rm viliov45.zip
!kaggle datasets download muennighoff/viliov90
!unzip -j viliov90.zip -d $model_dir/viliov90
!rm viliov90.zip
!kaggle datasets download muennighoff/viliov135
!unzip -j viliov135.zip -d $model_dir/viliov135
!rm viliov135.zip

# ES
# !kaggle datasets download muennighoff/vilioe36
# !unzip -j vilioe36.zip vilioes/step_2500train/* -d $model_dir/step_2500train
# !unzip -j vilioe36.zip vilioes/step_2500traindev/* -d $model_dir/step_2500traindev
# !rm vilioe36.zip

100% 787M/787M [00:39<00:00, 24.3MB/s]
100% 787M/787M [00:39<00:00, 20.8MB/s]
Archive:  viliov45.zip
  inflating: /vilio/ckpt/viliov45/LASTtrain.pth  
  inflating: /vilio/ckpt/viliov45/LASTtraindev.pth  
100% 786M/787M [00:43<00:00, 22.3MB/s]
100% 787M/787M [00:43<00:00, 18.9MB/s]
Archive:  viliov90.zip
  inflating: /vilio/ckpt/viliov90/LASTtrain.pth  
  inflating: /vilio/ckpt/viliov90/LASTtraindev.pth  
100% 787M/787M [00:39<00:00, 20.0MB/s]
100% 787M/787M [00:39<00:00, 20.8MB/s]
Archive:  viliov135.zip
  inflating: /vilio/ckpt/viliov135/LASTtrain.pth  
  inflating: /vilio/ckpt/viliov135/LASTtraindev.pth  


In [ ]:
!echo $model_dir
!echo =========================
!cd $model_dir;ls

/vilio/ckpt
viliov135  viliov45  viliov90


In [ ]:
print(f"Installation finished in {time.time()-t0} seconds")

Installation finished in 2814.3627655506134 seconds


In [ ]:
import os
os.kill(os.getpid(), 9)

### check [gpu version](https://blog.paperspace.com/alternative-to-google-colab-pro/)

In [ ]:
# https://stackoverflow.com/questions/64526139/how-does-one-get-the-model-of-the-gpu-in-python-and-save-it-as-a-string
import subprocess
import os
def get_mdl():
    line_as_bytes = subprocess.check_output("nvidia-smi -L", shell=True)
    line = line_as_bytes.decode("ascii")
    _, line = line.split(":", 1)
    line, _ = line.split("(")
    return line.strip()

In [ ]:
import time
gpu_type = get_mdl()
if "A100" in gpu_type:
  print(f"{gpu_type} detected, killing runtime. Rerun this section from the beginning again")
  time.sleep(5)
  os.kill(os.getpid(), 9)
else:
  print(f"GPU {gpu_type}: proceeding to next section")

GPU Tesla T4: proceeding to next section


In [ ]:
!ls -d /usr/local/cuda-*
!which nvcc

/usr/local/cuda-10.2  /usr/local/cuda-11  /usr/local/cuda-11.8
/usr/local/cuda/bin/nvcc


In [ ]:
p = os.getenv('PATH')
ld = os.getenv('LD_LIBRARY_PATH')
os.environ['PATH'] = f"/usr/local/cuda-10.2/bin:{p}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-10.2/lib64:{ld}"
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Oct_23_19:24:38_PDT_2019
Cuda compilation tools, release 10.2, V10.2.89


### inference

In [ ]:
model_dir = "/vilio/ckpt"

In [1]:
# !echo $model_dir
# # # O
# # !cd /vilio; bash ./bash/inference/O/hm_O.sh \
# #     $model_dir/vilioov50/LASTtrain.pth \
# #     $model_dir/vilioo50/LASTtrain.pth \
# #     $model_dir/vilioo36/LASTtrain.pth
# # !cd /vilio; bash ./bash/inference/O/hm_O_correct_label.sh \
# #     $model_dir/vilioov50/LASTtrain.pth \
# #     $model_dir/vilioo50/LASTtrain.pth \
# #     $model_dir/vilioo36/LASTtrain.pth
# # !cd /vilio/bash; bash ./micace_evaluator.sh \
# #     /content/drive/MyDrive/vilio/export/O O
# # !cd /vilio/bash; bash ./micace_evaluator.sh \
# #     /content/drive/MyDrive/vilio/export/correct_label/O O
# # # U
# # !cd /vilio; bash ./bash/inference/U/hm_U.sh \
# #     $model_dir/viliou72/LASTtrain.pth \
# #     $model_dir/viliou50/LASTtrain.pth \
# #     $model_dir/viliou36/LASTtrain.pth
# # !cd /vilio; bash ./bash/inference/U/hm_U_correct_label.sh \
# #     $model_dir/viliou72/LASTtrain.pth \
# #     $model_dir/viliou50/LASTtrain.pth \
# #     $model_dir/viliou36/LASTtrain.pth
# # !cd /vilio/bash; bash ./micace_evaluator.sh \
# #     /content/drive/MyDrive/vilio/export/U U
# # !cd /vilio/bash; bash ./micace_evaluator.sh \
# #     /content/drive/MyDrive/vilio/export/correct_label/U U
# V
!cd /vilio; bash ./bash/inference/V/hm_V.sh \
    $model_dir/viliov45/LASTtrain.pth \
    $model_dir/viliov90/LASTtrain.pth \
    $model_dir/viliov135/LASTtrain.pth
!cd /vilio; bash ./bash/inference/V/hm_V_correct_label.sh \
    $model_dir/viliov45/LASTtrain.pth \
    $model_dir/viliov90/LASTtrain.pth \
    $model_dir/viliov135/LASTtrain.pth
# # !cd /vilio/bash; bash ./micace_evaluator.sh \
# #     /content/drive/MyDrive/vilio/export/V V
# # !cd /vilio/bash; bash ./micace_evaluator.sh \
# #     /content/drive/MyDrive/vilio/export/correct_label/V V
# # ES
# # !cd /vilio/ernie-vil; bash ./bash/inference/ES/hm_ES36.sh \
# #                             /vilio/ckpt/step_2500train \
# #                             /vilio/ckpt/step_2500traindev
# # !cd /vilio/bash; bash ./micace_evaluator.sh \
# #     /content/drive/MyDrive/vilio/export/ES ES

# EOS